# Route between multiple Runnables

This notebook covers how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:

1. Using a `RunnableBranch`.
2. Writing custom factory function that takes the input of a previous step and returns a **runnable**. Importantly, this should return a **runnable** and NOT actually execute.

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about `LangChain`, `Anthropic`, or `Other`, then routes to a corresponding prompt chain.

## Using a RunnableBranch

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input. 

If no provided conditions match, it runs the default runnable.

Here's an example of what it looks like in action:

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser

First, let's create a chain that will identify incoming questions as being about `LangChain`, `Anthropic`, or `Other`:

In [2]:
chain = PromptTemplate.from_template("""Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:""") | ChatAnthropic() | StrOutputParser()

In [3]:
chain.invoke({"question": "how do I call Anthropic?"})

' Anthropic'

Now, let's create three sub chains:

In [4]:
langchain_chain = PromptTemplate.from_template("""You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()
anthropic_chain = PromptTemplate.from_template("""You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()
general_chain = PromptTemplate.from_template("""Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()

In [5]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
  (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
  (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
  general_chain
)

In [6]:
full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | branch

In [7]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content=" As Dario Amodei told me, here are a few ways to use Anthropic:\n\n- Sign up for an account at Anthropic.com to access the free tier of Claude, Anthropic's conversational AI assistant. You can chat with Claude through the website.\n\n- Integrate Claude into your own products and services using Anthropic's API. This allows you to leverage Claude's natural language capabilities in your own applications.\n\n- Use Anthropic PBC, their proprietary AI safety technique, to align Claude's responses with your values and intended use cases. PBC helps ensure reliable and safe conversations.\n\n- Leverage their self-supervised and constitutional AI techniques like Constitutional AI and Self-Supervised Learning to train AI systems that are helpful, harmless, and honest.\n\n- Apply for partnerships if you are working on particularly useful or important applications of AI conversational agents. Anthropic selectively collaborates this way.\n\n- Follow Anthropic's research publicatio

In [8]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content=" As Harrison Chase told me, here is how you use LangChain:\n\nLangChain is an AI assistant that allows you to have natural conversations and ask questions on any topic. To use LangChain, simply type or speak your question or request just as you would to a friend. LangChain will then generate a helpful response to continue the conversation. \n\nYou can ask LangChain open-ended questions, request definitions or explanations on a concept, have it summarize information for you, and more. The key is to engage with it in a natural, conversational way. Don't worry about using special commands or syntax. Over time and with more conversations, LangChain will learn your interests and communication style to provide increasingly relevant and personalized responses.\n\nSome examples of questions you can ask LangChain:\n\n- What is the capital of France?\n- Can you explain the key principles of machine learning in simple terms?  \n- What is the plot summary of the book X?\n- What 

In [9]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content=' 4', additional_kwargs={}, example=False)

## Using a custom function

You can also use a custom function to route between different outputs. Here's an example:

In [10]:
def route(info):
    inputs = {"question": lambda x: x["question"]}
    if "anthropic" in info["topic"].lower():
        return inputs | anthropic_chain
    elif "langchain" in info["topic"].lower():
        return inputs | langchain_chain
    else:
        return inputs | general_chain

In [11]:
from langchain.schema.runnable import RunnableLambda

full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | RunnableLambda(route)

In [12]:
full_chain.invoke({"question": "how do I use Anthroipc?"})

AIMessage(content=" As Dario Amodei told me, to use Anthropic's conversational AI system Anthro IPC, you first need to sign up for an API key at anthropic.com. Once you have a key, you can make requests to the Anthro IPC API endpoints to have conversations with the AI assistant. The basic process is:\n\n1. Send a POST request to the `/message` endpoint with your API key and the user message as the request body. This will return the AI's response.\n\n2. To have a conversation, continue sending requests to the `/message` endpoint, each time including the previous messages and responses in the request body. The AI will use the context to understand the ongoing conversation.\n\n3. Make sure to implement rate limiting on requests as per Anthropic's usage guidelines. The free tier has limits on requests per minute and month.\n\n4. Refer to the Anthro IPC documentation for details on request parameters, endpoints, error handling, etc. There are also client libraries available to make integrat

In [13]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content=" As Harrison Chase told me, here is how you can use LangChain:\n\nLangChain is an AI assistant that can have natural conversations and provide helpful information to users. To start a conversation with LangChain, simply ask it a question or make a request. LangChain will then try to understand what you are asking and provide a relevant and thoughtful response. \n\nYou can ask LangChain questions on a wide variety of topics - from seeking advice, to asking for definitions of words, to having a casual chat. LangChain aims to have human-like conversations and be helpful, harmless, and honest in its responses. \n\nSome examples of questions you could ask LangChain:\n\n- What's the weather forecast for tomorrow?\n- Can you recommend a good book to read? \n- What is the capital of Australia?\n- How do you make lasagna?\n- What is your favorite color?\n\nLangChain may sometimes admit if it does not know the answer to a question. But it will try its best to provide useful in

In [14]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content=' 4', additional_kwargs={}, example=False)